In [ ]:
import numpy as np
import pandas as pd
import time
import os
import matplotlib.pyplot as plt
# import dwf

from ctypes import *

%load_ext autoreload
%autoreload 2

In [ ]:
hex(0b1101000)

In [ ]:
hex(225)

In [ ]:
import sys
sys.platform

In [ ]:
from chip_systems import AD2
from chip_systems import ProjectSystem
from chip_systems import Biosensors

In [ ]:
interface = AD2.AD2_I2C(voltage=3.3, pin=0)

In [ ]:
device = Biosensors.MAX30101(interface=interface)

In [ ]:
device.ReadTemperature()

In [ ]:
interface.SetAnalogOut(0, 0)

In [ ]:
iNak = c_int()
rgRX = (c_ubyte*6)()

hdwf = c_int()

dwf = cdll.LoadLibrary("dwf.dll")

class ad2():
    
    def __init__(self):
        self.dwf = dwf
        self.dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
    
    def set_analog_out(self, voltage, pin=1):
        
        self.dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(pin), c_int(0), c_double(True))  # enable positive supply
        self.dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(pin), c_int(1), c_double(voltage))
        self.dwf.FDwfAnalogIOEnableSet(hdwf, c_int(True))  # master enable
        
    def setup_i2c(self, scl_pin, sda_pin):
        
        # set I2C frequency and pins
        self.dwf.FDwfDigitalI2cRateSet(hdwf, c_double(1e5)) # 100kHz

        # pin numbers need to be checked for use with AD or AD2 device
        self.dwf.FDwfDigitalI2cSclSet(hdwf, c_int(scl_pin - 24))
        self.dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(sda_pin - 24))

        # initialize I2C (check for correct pull-ups)
        self.dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))
        if iNak.value == 0:
            print('I2C bus error. Check the pull-ups')
        time.sleep(0.1)
                
    def read_i2c(self, address, register):
        rgRX = (c_ubyte*6)()
        self.dwf.FDwfDigitalI2cWriteRead(hdwf, c_int(address << 1), (c_ubyte*1)(register), c_int(1), rgRX, c_int(1), byref(iNak))
        return rgRX[0]
    
    def write_i2c(self, address, register, value):
        self.dwf.FDwfDigitalI2cWrite(hdwf, c_int(address << 1), (c_ubyte*2)(register, value), c_int(2), byref(iNak))
        
    def close(self):
        # disable power supply
        self.dwf.FDwfAnalogIOEnableSet(hdwf, c_int(False))

        # disconnect all connected devices
        self.dwf.FDwfDeviceCloseAll()